In [3]:
# IMDB movie review with 50k
!pip install spacy

     ---------------------------------------- 12.3/12.3 MB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 48.9/48.9 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 94.7/94.7 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 2.3 MB/s eta 0:00:00
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
     -------------------------------------- 45.9/45.9 kB 757.9 kB/s eta 0:00:00
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
     -------------------------------------- 480.9/480.9 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 1.7 MB/s eta 0:00:00


In [5]:
!pip install wordcloud

     ------------------------------------ 152.1/152.1 kB 824.7 kB/s eta 0:00:00


In [9]:
!pip install textblob

  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)


In [10]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import sklearn 
import nltk 
import spacy
import re 
import string
import unicodedata
import os 
import warnings
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud , STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize , sent_tokenize 
from bs4 import BeautifulSoup 
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer 
from sklearn.linear_model import LogisticRegression , SGDClassifier 
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob 
from textblob import Word 
from sklearn.metrics import classification_report , confusion_matrix ,accuracy_score 

warnings.filterwarnings('ignore')


In [13]:
### read data 
path  = 'E:\projects\sentiment analysis\IMDB_data\IMDB Dataset.csv'
data  = pd.read_csv(path)

In [16]:
data.tail(10)

,review,sentiment
49990,"Lame, lame, lame!!! A 90-minute cringe-fest th...",negative
49991,"Les Visiteurs, the first movie about the medie...",negative
49992,John Garfield plays a Marine who is blinded by...,positive
49993,Robert Colomb has two full-time jobs. He's kno...,negative
49994,This is your typical junk comedy.<br /><br />T...,negative
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [17]:
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [18]:
## data is balanced 
data.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [20]:
#now time for preprocessing 

#let's do some steps 

#1. remove HTML 
#2. remove squer prackets 
#3. remove special characters 
#4. remove stopwords 
#5. stemming 

# finally collect all functions in one preprocessing function 


def remove_html(text):
    soup = BeautifulSoup(text , 'html.parser')
    return soup.get_text()

def remove_squer_prackets(text):
    return re.sub('\[[^]]*\]','',text)

def remove_special_char(text):
    return re.sub('[^a-zA-Z0-9\s]','' , text)

def stemming(text):
    stem = nltk.porter.PorterStemmer()
    text = ' '.join([stem.stem(word) for word in text.split()])
    return text 

def remove_stopwords(text):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    filtering = [word for word in tokens if word.lower() not in stopwords]
    return ' '.join(filtering)

# collecte
def preprocessing(text):
    docs = remove_html(text)
    docs = remove_squer_prackets(docs)
    docs = remove_special_char(docs)
    docs = stemming(docs)
    docs = remove_stopwords(docs)
    return docs


In [21]:
tokenizer = ToktokTokenizer()
stopwords = nltk.corpus.stopwords.words('english')


# call preprocessing function 

processed_data =  data['review'].apply(preprocessing)

In [34]:
# now let's split data into train and test data 
from sklearn.model_selection import train_test_split

label_encoder = LabelBinarizer()
labeled = label_encoder.fit_transform(data['sentiment'])


train_data, val_data , train_label , val_label =train_test_split(processed_data ,labeled, test_size=0.1 , shuffle=True )


train_label = pd.DataFrame(train_label)
val_label = pd.DataFrame(val_label)

In [37]:
# now we have two methode to train models 

# first ---> CountVectorizer   which mean Bag Of words  
# second --> TfidfVectorizer   which mean Teaem frequency inverse document frequency 


# let's try each of them 

countV = CountVectorizer(ngram_range = (1,3) ,binary = False , min_df = 0 , max_df=1 )

count_train = countV.fit_transform(train_data)
count_val   = countV.transform(val_data)


######################################


TFIDF_model = TfidfVectorizer(ngram_range=(1,3) , use_idf=True , min_df = 0 , max_df=1)

tf_train = TFIDF_model.fit_transform(train_data) 
tf_val   = TFIDF_model.transform(val_data)


In [41]:
# now let's build models to trian it in our data 
# let's show what we have 

# 1.Logistic Regression model 
# 2.Stochastic Gradient Descent classify model 
# 3.naive bayes model 


# init our model with default hyperparameters 
LR_model = LogisticRegression()
SGD_model= SGDClassifier()
NB_model = MultinomialNB()

# let's start with lgistic Regression 

LR_BOW  = LR_model.fit(count_train ,train_label)
LR_TFIDF= LR_model.fit(tf_train , train_label)


In [42]:
# now let's evaluate our model and test it 


BOW_prediction= LR_model.predict(count_val)
tf_prediction = LR_model.predict(tf_val)

BOW_accuracy= accuracy_score(BOW_prediction , val_label)
tf_accuracy = accuracy_score(tf_prediction , val_label)


print(f'accuracy from Logistic Regression model :\nBOW:\t{BOW_accuracy}\nTFIDF:\t{tf_accuracy}')


accuracy from Logistic Regression model :
BOW:	0.7534
TFIDF:	0.7546


In [61]:

NB_BOW  = NB_model.fit(count_train ,train_label)
NB_TFIDF= NB_model.fit(tf_train , train_label)


In [46]:
# now let's evaluate our model and test it 


BOW_prediction= NB_model.predict(count_val)
tf_prediction = NB_model.predict(tf_val)

BOW_accuracy= accuracy_score(BOW_prediction , val_label)
tf_accuracy = accuracy_score(tf_prediction , val_label)


print(f'accuracy from Naive Bayes model :\nBOW:\t{BOW_accuracy}\nTFIDF:\t{tf_accuracy}')


accuracy from Naive Bayes model :
BOW:	0.754
TFIDF:	0.7534


In [48]:
# as we say that naive bayes and logistic regression have the same accuracy 
# but naive bayes faster than logistic regression 

#let's show classification report 


clas_repo  = classification_report(BOW_prediction , val_label , target_names =['positive' , 'negative'])

print(clas_repo)

              precision    recall  f1-score   support

    positive       0.75      0.76      0.76      2518
    negative       0.75      0.75      0.75      2482

    accuracy                           0.75      5000
   macro avg       0.75      0.75      0.75      5000
weighted avg       0.75      0.75      0.75      5000



In [49]:
# let's doing some modifications in naive's hyperparamters and show result 


from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'alpha': [0.1, 0.5, 1.0],
    'fit_prior': [True, False],
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5)
grid_search.fit(count_train, train_label)
grid_search.fit(tf_train, train_label)
# Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [53]:
# let's show accuracy 


BOW_prediction = grid_search.predict(count_val)
TF_prediction  = grid_search.predict(tf_val)

BOW_accuracy = accuracy_score(BOW_prediction , val_label)

tf_accuracy = accuracy_score(TF_prediction ,val_label)
print(f'accuracy from Naive Bayes model :\nBOW:\t{BOW_accuracy}\nTFIDF:\t{tf_accuracy}')

accuracy from Naive Bayes model :
BOW:	0.7586
TFIDF:	0.7586
